In [16]:
from flowmason import conduct, MapReduceStep, SingletonStep, load_artifact # TODO: install this package from me: https://github.com/smfsamir/flowmason

# from allosaurus.app import read_recognizer
import panphon.distance # https://github.com/dmort27/panphon

# TODO: you'll have to install these packages. Let me know if there's any trouble here; you should be able to do `pip install` for all of them
import polars as pl
import torch
import soundfile as sf
import os
import ipdb
import zipfile
import os
from scipy.io import wavfile
import shutil
from praatio import textgrid as tgio
import re
import librosa
import time

from typing import List
from collections import OrderedDict
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC 



In [17]:

def get_root_path():
    cwd = os.getcwd();
    return cwd


HF_CACHE_DIR= get_root_path() + "\outputs\hf_cache_dir"
SCRATCH_DIR= get_root_path()+ "\outputs\scratch_dir"

In [20]:
arpabet_to_ipa = {
    'AA': 'ɑ', 
    'AE': 'æ',
    'AH': 'ʌ',
    'AO': 'ɔ',
    'AW': 'aʊ',
    'AY': 'aɪ',
    'EH': 'ɛ',
    'ER': 'ɝ',
    'EY': 'eɪ',
    'IH': 'ɪ',
    'IY': 'i',
    'OW': 'oʊ',
    'OY': 'ɔɪ',
    'UH': 'ʊ',
    'UW': 'u',
    'B': 'b',
    'CH': 'tʃ',
    'D': 'd',
    'DH': 'ð',
    'F': 'f',
    'G': 'g',
    'HH': 'h',
    'JH': 'dʒ',
    'K': 'k',
    'L': 'l',
    'M': 'm',
    'N': 'n',
    'NG': 'ŋ',
    'P': 'p',
    'R': 'ɹ',
    'S': 's',
    'SH': 'ʃ',
    'T': 't',
    'TH': 'θ',
    'V': 'v',
    'W': 'w',
    'Y': 'j',
    'Z': 'z',
    'ZH': 'ʒ',
}


# def get_data_iterator():
#     cwd = os.getcwd();
#     print(cwd)
#     url = cwd + '\datasets\l2arctic_release_v5.0.zip';
#     iteratable = process_zip(url, 1)
#     return iteratable

def process_zip(zip_path):
    print('start processing arctic dataset:' + format(time.perf_counter()));
    timestart = time.perf_counter();
    difference = 0;
    nextstart = time.perf_counter();
    
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Extract all the contents into a temporary directory
        temp_dir = 'datasets'
        zip_ref.extractall(temp_dir)
        
        # Getting a list of zip files (each speaker is in a separate zip file)
        speaker_zips = [f for f in os.listdir(temp_dir) if f.endswith('.zip')]
        
        iteratable = []
        countforspeaker = 0
        
        for speaker_zip in speaker_zips:
            difference = time.perf_counter() - nextstart;
            nextstart = time.perf_counter();
            print('speaker'+ speaker_zip + ' : ' + format(difference));
            speaker_zip_path = os.path.join(temp_dir, speaker_zip)
            speaker_temp_dir = os.path.join(temp_dir, speaker_zip.replace('.zip', ''))
            # Extract the speaker's zip file
            with zipfile.ZipFile(speaker_zip_path, 'r') as speaker_zip_ref:
                speaker_zip_ref.extractall(speaker_temp_dir)
                
            # internal_dir = os.listdir(speaker_temp_dir)[0]
            # print(internal_dir)
            # wav_dir = os.path.join(speaker_temp_dir, internal_dir, 'wav')
            # #annotation_dir = os.path.join(speaker_temp_dir, internal_dir, 'annotation')
            # textgrid_dir = os.path.join(speaker_temp_dir, internal_dir, 'textgrid')
            # #transcript_dir = os.path.join(speaker_temp_dir, internal_dir, 'transcript')
            
            # if os.path.exists(wav_dir): wav_files = get_all_files(wav_dir)
            # #if os.path.exists(annotation_dir): annotation_files = get_all_files(annotation_dir)
            # if os.path.exists(textgrid_dir): textgrid_files = get_all_files(textgrid_dir)
            # #if os.path.exists(transcript_dir): transcript_files = get_all_files(transcript_dir)
            
            # countforsample = 0
            # for wav_file,  textgrid_file in zip(wav_files, textgrid_files):
            #     words, tphones = extract_words_and_phones(textgrid_file)
            #     # words, aphones = extract_words_and_phones(annotation_file)
            #     #list(map(lambda phone: phone[2], phones))
            #     print(words)
            #     print(tphones)
            #     #print(aphones)
                
            #     ipa_transcription = convert_arpabet_to_ipa(tphones)
                
            #     audio, sr = librosa.load(wav_file)
            #     #with open(transcript_file, 'r') as file:
            #     iteratable.append((audio, ipa_transcription))
                    
            #     countforsample+=1
            #     if (countforsample == numberofsampleperSpeaker):
            #         break
                    
            # # for wav_file in wav_files:
            # #     sample_rate, data = wavfile.read(wav_file)
            # #     #print(data)
            # #     audios.append(data)
            
            # countforspeaker+=1
            # if (numberofspeaker == countforspeaker):
            #     break
                
                
                
        
        # shutil.rmtree(temp_dir)
        difference = time.perf_counter() - timestart;
        
        print('finishing processing arctic dataset: ' + format(difference));
        
        # return iteratable
                
        
            

def get_all_files(directory):
    files_and_dirs = os.listdir(directory)
    
    files_paths = []
    files = [f for f in files_and_dirs if os.path.isfile(os.path.join(directory, f))]
    
    for file in files:
        files_paths.append(os.path.join(directory, file))
    
    return files_paths


def extract_words_and_phones(textgrid_path):
    # Load the TextGrid file
    tg = tgio.openTextgrid(textgrid_path, includeEmptyIntervals=False)

    # Assuming the names of the tiers are 'words' and 'phones'
    words_tier = tg.getTier('words')
    phones_tier = tg.getTier('phones')

    # Extract words and their intervals
    words = [(interval.start, interval.end, interval.label) for interval in words_tier.entries]

    # Extract phones and their intervals
    phones = [(interval.start, interval.end, re.sub(r'\d+', '', interval.label)) for interval in phones_tier.entries]

    return words, phones



def convert_arpabet_to_ipa(arpabet_phones):
    ipa_phones = [arpabet_to_ipa[phone[2]] for phone in arpabet_phones if phone[2] in arpabet_to_ipa]
    return ''.join(ipa_phones)


cwd = os.getcwd();
print(cwd)
url = cwd + '\datasets\l2arctic_release_v5.0.zip';
iteratable = process_zip(url)


c:\Users\david\OneDrive\Desktop\school\URO\URO_multilingual_speech_recognition
start processing arctic dataset:13685.7532307
speakerABA.zip : 20.947451800000636
speakerASI.zip : 7.639224499998818
speakerBWC.zip : 5.742717700000867
speakerEBVS.zip : 6.897728500000085
speakerERMS.zip : 5.812350300000617
speakerHJK.zip : 6.4035198999990826
speakerHKK.zip : 5.908732500000042
speakerHQTV.zip : 6.157685899999706
speakerl2arctic_release_v5.0.zip : 5.987224500000593
speakerLXC.zip : 20.447983600000953
speakerMBMPS.zip : 5.686578299999383
speakerNCC.zip : 5.5227562000000034
speakerNJS.zip : 5.228591499999311
speakerPNV.zip : 4.838991399999941
speakerRRBI.zip : 5.190570900000239
speakerSKA.zip : 5.079096100000243
speakersuitcase_corpus.zip : 4.318308500000057
speakerSVBI.zip : 0.974261999999726
speakerTHV.zip : 5.060335399999531
speakerTLV.zip : 5.177257199999076
speakerTNI.zip : 5.683974000001399
speakerTXHC.zip : 5.913738400000511
speakerYBAA.zip : 6.67678479999995
speakerYDCK.zip : 6.42539629

In [21]:
def get_data_iterator():
    cwd = os.getcwd();
    url = cwd + '\datasets';
    iteratable = process_datasets(url)
    return iteratable

def process_datasets(data_path, numberofspeaker = 1, numberofsampleperSpeaker = 1):
    print('start processing arctic dataset:' + format(time.perf_counter()));
    timestart = time.perf_counter();
    difference = 0;
    nextstart = time.perf_counter();
    
    # Get a list of all items in 'data_path'
    all_items = os.listdir(data_path)

    # Filter for directories only, excluding any zip files or other file types
    speaker_folders = [os.path.join(data_path, f) for f in all_items if os.path.isdir(os.path.join(data_path, f)) and not f.endswith('.zip')]
        
    iteratable = []
    countforspeaker = 0
   
        
    for speaker_folder in speaker_folders:
        difference = time.perf_counter() - nextstart;
        nextstart = time.perf_counter();
        print('speaker'+ speaker_folder + ' : ' + format(difference));
        internal_dir = os.listdir(speaker_folder)[0]
        print(internal_dir)
        wav_dir = os.path.join(speaker_folder, internal_dir, 'wav')
            
        textgrid_dir = os.path.join(speaker_folder, internal_dir, 'textgrid')
       
        if os.path.exists(wav_dir): wav_files = get_all_files(wav_dir)
            
        if os.path.exists(textgrid_dir): textgrid_files = get_all_files(textgrid_dir)
            
            
        
        countforsample = 0
        for wav_file,  textgrid_file in zip(wav_files, textgrid_files):
            words, tphones = extract_words_and_phones(textgrid_file)
            ipa_transcription = convert_arpabet_to_ipa(tphones)
                
            audio, sr = librosa.load(wav_file)
                
            iteratable.append((audio, ipa_transcription))
                    
            countforsample+=1
            if (countforsample == numberofsampleperSpeaker):
                break
        
            
        countforspeaker+=1
        if (numberofspeaker == countforspeaker):
            break
                
                
    difference = time.perf_counter() - timestart;
        
    print('finishing processing arctic dataset: ' + format(difference));
        
    return iteratable


result = get_data_iterator();
print(result)

start processing arctic dataset:13883.4263684
speakerc:\Users\david\OneDrive\Desktop\school\URO\URO_multilingual_speech_recognition\datasets\ABA : 0.0051937999996880535
ABA
finishing processing arctic dataset: 0.34489250000115135
[(array([ 1.1901691e-06, -6.8628915e-06,  1.5418029e-05, ...,
        7.9195611e-03,  7.6785325e-03,  1.0348516e-02], dtype=float32), 'ɔθɝʌvðʌdeɪndʒɝtɹeɪlfɪlɪpstilzɛtsɛtɝʌ')]


In [22]:
# generate predictions
def step_generate_pfer(prediction_frame: pl.DataFrame, 
                                  **kwargs):
    feat_edit_distance = panphon.distance.Distance().feature_edit_distance
    # currently, the predictions have spaces between segments. Remove the spaces
    # in the predictions column
    prediction_frame = prediction_frame.with_columns([
        pl.col('predictions').map_elements(lambda x: x.replace(' ', '')).alias('predictions')
    ])
    prediction_frame = prediction_frame.with_columns([
        pl.struct(['predictions', 'transcripts']).map_elements(
            lambda x: feat_edit_distance(x['predictions'], x['transcripts'])).alias('pfer')
    ])
    # group by the language code and compute the average pfer
    return prediction_frame

In [23]:
def step_generate_predictions_notre_dame(**kwargs) -> str:
    model = Wav2Vec2ForCTC.from_pretrained("ctaguchi/wav2vec2-large-xlsr-japlmthufielta-ipa1000-ns", cache_dir=HF_CACHE_DIR)
    processor = Wav2Vec2Processor.from_pretrained("ctaguchi/wav2vec2-large-xlsr-japlmthufielta-ipa1000-ns", cache_dir=HF_CACHE_DIR)
    predictions = []
    transcripts = []
    for datapoint in get_data_iterator(): # TODO: you have to implement this function to iterate over the arctic samples
        audio, txt = datapoint # TODO: the iterator should returns tuples of the audio array and the transcript
        #print(audio)
        input_values = processor(audio, return_tensors="pt", sampling_rate=16000).input_values
        # Convert input_values to double
        #input_values_double = input_values.double()

        # Ensure the model is in double precision
        #model = model.double()  
        with torch.no_grad():
            logits = model(input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        prediction = processor.batch_decode(predicted_ids)[0]
        predictions.append(prediction)
        transcripts.append(txt)
        
    print(pl.DataFrame({
        "predictions": predictions,
        "transcripts": transcripts
    }))
    
    print(predictions);
    print(transcripts);
    return pl.DataFrame({
        "predictions": predictions,
        "transcripts": transcripts
    })


result = step_generate_predictions_notre_dame()

print(result)

Some weights of the model checkpoint at ctaguchi/wav2vec2-large-xlsr-japlmthufielta-ipa1000-ns were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at ctaguchi/wav2vec2-large-xlsr-japlmthufielta-ipa1000-ns and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.we

start processing arctic dataset:13899.929499
speakerc:\Users\david\OneDrive\Desktop\school\URO\URO_multilingual_speech_recognition\datasets\ABA : 0.004579200000080164
ABA
finishing processing arctic dataset: 0.13926989999890793
shape: (1, 2)
┌──────────────────────────────────┬───────────────────────────────────┐
│ predictions                      ┆ transcripts                       │
│ ---                              ┆ ---                               │
│ str                              ┆ str                               │
╞══════════════════════════════════╪═══════════════════════════════════╡
│ jisuɾuvgudynd͡ʒuɾtɾojomfulibisti… ┆ ɔθɝʌvðʌdeɪndʒɝtɹeɪlfɪlɪpstilzɛts… │
└──────────────────────────────────┴───────────────────────────────────┘
['jisuɾuvgudynd͡ʒuɾtɾojomfulibistilinztstɾu']
['ɔθɝʌvðʌdeɪndʒɝtɹeɪlfɪlɪpstilzɛtsɛtɝʌ']
shape: (1, 2)
┌──────────────────────────────────┬───────────────────────────────────┐
│ predictions                      ┆ transcripts                       │

In [15]:
if __name__ == "__main__":
    step_dict = OrderedDict()

    step_dict['step_generate_preds_notre_dame'] = SingletonStep(step_generate_predictions_notre_dame, {
        "version": "001"
    })
    
    # step_dict['step_generate_pfer_notre_dame'] = SingletonStep(step_generate_pfer, {
    #     'version': '001', 
    #     'prediction_frame': 'map_step_generate_notredame_preds'
    # })
    metadata = conduct(os.path.join(SCRATCH_DIR, "arctic_flowmason_cache"), step_dict, "arctic_experiment_logs")
    ipdb.set_trace()

2024-02-23 14:00:31.190 | INFO     | flowmason.dag:conduct:314 - Step step_generate_preds_notre_dame is cached at C:\Users\david\OneDrive\Desktop\school\URO\URO_multilingual_speech_recognition\outputs\scratch_dir\arctic_flowmason_cache\d048277d42c6db1a98a4ff070727e347eaf6fccfc96700e07a1641e531b464b4, continuing.


--Return--
None
> c:\users\david\appdata\local\temp\ipykernel_12192\2046869975.py(13)<module>()

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
